# ML.Net - ModelExplainability

In [1]:
// ML.NET Nuget packages installation
#r "nuget:Microsoft.ML" 

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package Microsoft.ML version 1.5.0

## Using C# Class

In [2]:
using Microsoft.ML;
using Microsoft.ML.Transforms.Text;
using Microsoft.ML.Data;
using System;
using System.Collections.Generic;

## Declare data-classes for input data and predictions

In [3]:
public class HousingData
{
    [LoadColumn(0)]
    public float Longitude { get; set; }

    [LoadColumn(1)]
    public float Latitude { get; set; }

    [LoadColumn(2)]
    public float HousingMedianAge { get; set; }

    [LoadColumn(3)]
    public float TotalRooms { get; set; }

    [LoadColumn(4)]
    public float TotalBedrooms { get; set; }

    [LoadColumn(5)]
    public float Population { get; set; }

    [LoadColumn(6)]
    public float Households { get; set; }

    [LoadColumn(7)]
    public float MedianIncome { get; set; }

    [LoadColumn(8), ColumnName("Label")]
    public float MedianHouseValue { get; set; }

    [LoadColumn(9)]
    public string OceanProximity { get; set; }
}

public class HousingPrediction
{
    [ColumnName("Score")]
    public float PredictedHouseValue { get; set; }

    public float[] FeatureContributions { get; set; }
}

## Predict

In [4]:
var context = new MLContext();

var data = context.Data.LoadFromTextFile<HousingData>("./Datasets/housing/housing.csv", hasHeader: true, separatorChar: ',');

var features = data.Schema
    .Select(col => col.Name)
    .Where(colName => colName != "Label" && colName != "OceanProximity")
    .ToArray();

var pipeline = context.Transforms.Text.FeaturizeText("Text", "OceanProximity")
    .Append(context.Transforms.Concatenate("Features", features))
    .Append(context.Regression.Trainers.LbfgsPoissonRegression());

var model = pipeline.Fit(data);
var transformedData = model.Transform(data);

// Get weights of model
var linearModel = model.LastTransformer.Model;

var weights = linearModel.Weights;

Console.WriteLine("Weights:");

for (int i = 0; i < features.Length; i++)
{
    Console.WriteLine($"Feature {features[i]} has weight {weights[i]}");
}

Console.WriteLine(Environment.NewLine);

// Get global feature importance
var lastTransformer = model.LastTransformer;

var featureImportance = context.Regression.PermutationFeatureImportance(lastTransformer, transformedData);

Console.WriteLine("Global feature importance:");
for (int i = 0; i < featureImportance.Count(); i++)
{
    Console.WriteLine($"Feature - {features[i]}: Difference in RMS - {featureImportance[i].RootMeanSquaredError.Mean}");
}

Console.WriteLine(Environment.NewLine);

// Get feature importance for each row
var firstRow = model.Transform(context.Data.TakeRows(transformedData, 1));

var featureContribution = context.Transforms.CalculateFeatureContribution(lastTransformer, normalize: false);

var featureContributionTransformer = featureContribution.Fit(firstRow);

var featureContributionPipeline = model.Append(featureContributionTransformer);

var predictionEngine = context.Model.CreatePredictionEngine<HousingData, HousingPrediction>(featureContributionPipeline);

var sampleData = new HousingData
{
    Longitude = -122.25f,
    Latitude = 37.85f,
    HousingMedianAge = 55.0f,
    TotalRooms = 1627.0f,
    TotalBedrooms = 235.0f,
    Population = 322.0f,
    Households = 120.0f,
    MedianIncome = 8.3014f,
    OceanProximity = "NEAR BAY"
};

var prediction = predictionEngine.Predict(sampleData);

Console.WriteLine("Row feature importance:");

for (int i = 0; i < prediction.FeatureContributions.Length; i++)
{
    Console.WriteLine($"Feature {features[i]} has feature contribution of {prediction.FeatureContributions[i]}");
}

Weights:
Feature Longitude has weight -0,1470739
Feature Latitude has weight -0,17262222
Feature HousingMedianAge has weight 0,005542431
Feature TotalRooms has weight 1,2958609E-05
Feature TotalBedrooms has weight -2,2897724E-05
Feature Population has weight -0,00026471904
Feature Households has weight 0,00077238877
Feature MedianIncome has weight 0,14026381


Global feature importance:
Feature - Longitude: Difference in RMS - 57912,17459680204
Feature - Latitude: Difference in RMS - 62996,05873808067
Feature - HousingMedianAge: Difference in RMS - 2527,627967636756
Feature - TotalRooms: Difference in RMS - 418,1821196783858
Feature - TotalBedrooms: Difference in RMS - 168,30830551184772
Feature - Population: Difference in RMS - 208433,2862401039
Feature - Households: Difference in RMS - 153662,9427716253
Feature - MedianIncome: Difference in RMS - 48435,85134851326


Row feature importance:
Feature Longitude has feature contribution of 17,979784
Feature Latitude has feature contributi